# RCS Benchmark

## Getting Started

You need [`pyqrack`](https://pypi.org/project/pyqrack/) to run this notebook.

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

In the Python package itself, there should be an executable called `qrack_cl_precompile`. This "pre-compiles" the OpenCL "just-in-time" ("JIT") device program, for your system's accelerators. You might want to find and run this utility first, to avoid the need to "recompile" the OpenCL device program every time you first load Qrack into your environment.

## Configuration

In [2]:
# Lowest qubit width
low = 8
# Highest qubit width
high = 28
# Samples to average per qubit width
samples = 1

In [3]:
import math
import os
import random
import time

from pyqrack import QrackSimulator

# For more specific details about all available Qrack environment variables,
# See the C++ repository README: https://github.com/unitaryfund/qrack

# This is the maximum total number of fully-entangled qubits you expect to fit in general RAM.
os.environ['QRACK_MAX_CPU_QB']='32'

# Above this threshold, "QTensorNetwork" restricts simulations to "past light cone."
# At or below the threshold, much more work can be reused.
os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='-1'

# These below are approximation options. (By default, Qrack simulates in the "ideal.")

# This is a number between "0" ("ideal") and "1" ("round to exactly Clifford") for near-Clifford rounding.
# os.environ['QRACK_NONCLIFFORD_ROUNDING_THRESHOLD']='1

# This is a number between "0" ("ideal") and "1" ("destroy all entanglement") for "SDRP,"
# "Schmidt decomposition rounding parameter". (https://arxiv.org/abs/2304.14969)
# os.environ['QRACK_QUNIT_SEPARABILITY_THRESHOLD']='0.6'

# This is a number between "0" ("ideal") and "1" ("combine all binary decision tree branches")
# that sets the allowable "epsilon" between "QBdt" branches to consider them equal.
# os.environ['QRACK_QBDT_SEPARABILITY_THRESHOLD']='0.0001'

2024-02-21 12:24:17.857674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Run the Benchmark

In [4]:
def bench_qrack(n):
    # This is basically a "quantum volume" (random) circuit.
    start = time.perf_counter()

    sim = QrackSimulator(n)

    lcv_range = range(n)
    all_bits = list(lcv_range)

    single_count = 0
    double_count = 0
    for _ in lcv_range:
        # Single-qubit gates
        for i in lcv_range:
            sim.u(i, random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi))

        # 2-qubit couplers
        unused_bits = all_bits.copy()
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            sim.mcx([unused_bits.pop()], unused_bits.pop())

    fidelity = sim.get_unitary_fidelity()
    # Terminal measurement
    # print(sim.m_all())
    sim.m_all()

    return (time.perf_counter() - start, fidelity)

# Make sure the OpenCL environment is initialized before timing.
# (You probably also want to precompile OpenCL kernels with the `qrack_cl_precompile` utility, in general.)
bench_qrack(1)

time_results = {}
fidelity_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_qrack(n))

    time_results[n] = sum(r[0] for r in width_results) / samples
    fidelity_results[n] = sum(r[1] for r in width_results) / samples
    print(n, ": ", time_results[n], " seconds, ", fidelity_results[n], " out of 1.0 fidelity")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_2070_Super.ir
8 :  0.0014786869978706818  seconds,  1.0  out of 1.0 fidelity
9 :  0.001409466000040993  seconds,  1.0  out of 1.0 fidelity
10 :  0.0025243339987355284  seconds,  1.0  out of 1.0 fidelity
11 :  0.004116194002563134  seconds,  1.0  out of 1.0 fidelity
12 :  0.004898246999800904  seconds,  1.0  out of 1.0 fidelity
13 :  0.00531630600016797  seconds,  1.0  out of 1.0 fidelity
14 :  0.008587918997363886  seconds,  1.0  out of 1.0 fidelity
15 :  0.008280185000330675  seconds,  1.0  out of 1.0 fidelity
16 :  0.009733590999530861  seconds,  1.0  out of 1.0 fidelity
17 :  0.010935603997495491  seconds,  1.0  out of 1.0 fidelity
18 :  0.013509694999811472  seconds,  1.0  out of 1.0 fidelity
19 :  0.023049732997606043  seconds,  1.0  out of 1.0 fidelity
20 :  0.04174166699885973  seconds,  1.0  out of 1.0 fidelity
21 :  0.07807361899904208  seconds,  1.0  out of 1.0 fidelity
22 :  0.16472775299916975